# Load the pymoo result object saved with pickle.
ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

## Load ctaea results

In [2]:
with open('res_bc_Cogent_ctaea.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_ctaea.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_ctaea.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_ctaea.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_ctaea.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_ctaea.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_ctaea.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_ctaea.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_ctaea.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_ctaea.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_ctaea.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_ctaea.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [3]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('ctaea (Cogent): ' + str(i_Cogent) + '\n')
    f.write('ctaea (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('ctaea (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('ctaea (Colt): ' + str(i_Colt) + '\n')
    f.write('ctaea (Funet): ' + str(i_Funet) + '\n')
    f.write('ctaea (Abvt): ' + str(i_Abvt) + '\n')
    f.write('ctaea (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('ctaea (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('ctaea (Internode): ' + str(i_Internode) + '\n')
    f.write('ctaea (Missouri): ' + str(i_Missouri) + '\n')
    f.write('ctaea (Ion): ' + str(i_Ion) + '\n')
    f.write('ctaea (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('ctaea (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('ctaea (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('ctaea (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('ctaea (Colt): ' + str(n_Colt) + '\n')
    f1.write('ctaea (Funet): ' + str(n_Funet) + '\n')
    f1.write('ctaea (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('ctaea (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('ctaea (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('ctaea (Internode): ' + str(n_Internode) + '\n')
    f1.write('ctaea (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('ctaea (Ion): ' + str(n_Ion) + '\n')
    f1.write('ctaea (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For ctaea method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For ctaea method: 
Ideal point:  [2.00000000e+00 3.00000000e+00 5.40731657e+01 9.74239350e-03]
Nadir point:  [9.90000000e+01 5.00000000e+01 9.56751065e+02 5.62449763e-01]


## Load dnsga2 results

In [4]:
with open('res_bc_Cogent_dnsga2.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_dnsga2.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_dnsga2.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_dnsga2.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_dnsga2.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_dnsga2.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_dnsga2.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_dnsga2.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_dnsga2.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_dnsga2.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_dnsga2.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_dnsga2.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [5]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('dnsga2 (Cogent): ' + str(i_Cogent) + '\n')
    f.write('dnsga2 (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('dnsga2 (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('dnsga2 (Colt): ' + str(i_Colt) + '\n')
    f.write('dnsga2 (Funet): ' + str(i_Funet) + '\n')
    f.write('dnsga2 (Abvt): ' + str(i_Abvt) + '\n')
    f.write('dnsga2 (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('dnsga2 (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('dnsga2 (Internode): ' + str(i_Internode) + '\n')
    f.write('dnsga2 (Missouri): ' + str(i_Missouri) + '\n')
    f.write('dnsga2 (Ion): ' + str(i_Ion) + '\n')
    f.write('dnsga2 (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('dnsga2 (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('dnsga2 (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('dnsga2 (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('dnsga2 (Colt): ' + str(n_Colt) + '\n')
    f1.write('dnsga2 (Funet): ' + str(n_Funet) + '\n')
    f1.write('dnsga2 (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('dnsga2 (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('dnsga2 (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('dnsga2 (Internode): ' + str(n_Internode) + '\n')
    f1.write('dnsga2 (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('dnsga2 (Ion): ' + str(n_Ion) + '\n')
    f1.write('dnsga2 (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For dnsga2 method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For dnsga2 method: 
Ideal point:  [0.00000000e+00 0.00000000e+00 5.62054493e+01 4.06267709e-03]
Nadir point:  [177.  67.  inf  inf]


## Load kgbdmoea results

In [6]:
with open('res_bc_Cogent_kgbdmoea.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_kgbdmoea.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_kgbdmoea.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_kgbdmoea.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_kgbdmoea.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_kgbdmoea.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_kgbdmoea.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_kgbdmoea.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_kgbdmoea.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_kgbdmoea.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_kgbdmoea.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_kgbdmoea.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [7]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('kgbdmoea (Cogent): ' + str(i_Cogent) + '\n')
    f.write('kgbdmoea (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('kgbdmoea (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('kgbdmoea (Colt): ' + str(i_Colt) + '\n')
    f.write('kgbdmoea (Funet): ' + str(i_Funet) + '\n')
    f.write('kgbdmoea (Abvt): ' + str(i_Abvt) + '\n')
    f.write('kgbdmoea (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('kgbdmoea (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('kgbdmoea (Internode): ' + str(i_Internode) + '\n')
    f.write('kgbdmoea (Missouri): ' + str(i_Missouri) + '\n')
    f.write('kgbdmoea (Ion): ' + str(i_Ion) + '\n')
    f.write('kgbdmoea (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('kgbdmoea (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('kgbdmoea (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('kgbdmoea (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('kgbdmoea (Colt): ' + str(n_Colt) + '\n')
    f1.write('kgbdmoea (Funet): ' + str(n_Funet) + '\n')
    f1.write('kgbdmoea (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('kgbdmoea (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('kgbdmoea (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('kgbdmoea (Internode): ' + str(n_Internode) + '\n')
    f1.write('kgbdmoea (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('kgbdmoea (Ion): ' + str(n_Ion) + '\n')
    f1.write('kgbdmoea (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For kgbdmoea method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For kgbdmoea method: 
Ideal point:  [0.00000000e+00 0.00000000e+00 5.41710479e+01 4.21019526e-03]
Nadir point:  [163.  73.  inf  inf]


## Load moead results

In [8]:
with open('res_bc_Cogent_moead.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_moead.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_moead.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_moead.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_moead.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_moead.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_moead.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_moead.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_moead.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_moead.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_moead.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_moead.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [9]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('moead (Cogent): ' + str(i_Cogent) + '\n')
    f.write('moead (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('moead (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('moead (Colt): ' + str(i_Colt) + '\n')
    f.write('moead (Funet): ' + str(i_Funet) + '\n')
    f.write('moead (Abvt): ' + str(i_Abvt) + '\n')
    f.write('moead (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('moead (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('moead (Internode): ' + str(i_Internode) + '\n')
    f.write('moead (Missouri): ' + str(i_Missouri) + '\n')
    f.write('moead (Ion): ' + str(i_Ion) + '\n')
    f.write('moead (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('moead (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('moead (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('moead (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('moead (Colt): ' + str(n_Colt) + '\n')
    f1.write('moead (Funet): ' + str(n_Funet) + '\n')
    f1.write('moead (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('moead (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('moead (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('moead (Internode): ' + str(n_Internode) + '\n')
    f1.write('moead (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('moead (Ion): ' + str(n_Ion) + '\n')
    f1.write('moead (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For moead method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For moead method: 
Ideal point:  [7.00000000e+00 2.00000000e+00 5.73401224e+01 2.14450400e-02]
Nadir point:  [4.50000000e+01 6.00000000e+00 4.79792713e+02 1.33567032e-01]


## Load agemoea results

In [10]:
with open('res_bc_Cogent_agemoea.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_agemoea.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_agemoea.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_agemoea.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_agemoea.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_agemoea.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_agemoea.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_agemoea.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_agemoea.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_agemoea.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_agemoea.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_agemoea.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [11]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('agemoea (Cogent): ' + str(i_Cogent) + '\n')
    f.write('agemoea (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('agemoea (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('agemoea (Colt): ' + str(i_Colt) + '\n')
    f.write('agemoea (Funet): ' + str(i_Funet) + '\n')
    f.write('agemoea (Abvt): ' + str(i_Abvt) + '\n')
    f.write('agemoea (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('agemoea (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('agemoea (Internode): ' + str(i_Internode) + '\n')
    f.write('agemoea (Missouri): ' + str(i_Missouri) + '\n')
    f.write('agemoea (Ion): ' + str(i_Ion) + '\n')
    f.write('agemoea (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('agemoea (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('agemoea (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('agemoea (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('agemoea (Colt): ' + str(n_Colt) + '\n')
    f1.write('agemoea (Funet): ' + str(n_Funet) + '\n')
    f1.write('agemoea (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('agemoea (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('agemoea (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('agemoea (Internode): ' + str(n_Internode) + '\n')
    f1.write('agemoea (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('agemoea (Ion): ' + str(n_Ion) + '\n')
    f1.write('agemoea (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For agemoea method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For agemoea method: 
Ideal point:  [2.00000000e+00 3.00000000e+00 5.30564590e+01 6.67422622e-03]
Nadir point:  [1.46000000e+02 3.30000000e+01 1.59670616e+03 8.55748626e-01]


## Load agemoea2 results

In [12]:
with open('res_bc_Cogent_agemoea2.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_agemoea2.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_agemoea2.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_agemoea2.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_agemoea2.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_agemoea2.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_agemoea2.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_agemoea2.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_agemoea2.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_agemoea2.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_agemoea2.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_agemoea2.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [13]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('agemoea2 (Cogent): ' + str(i_Cogent) + '\n')
    f.write('agemoea2 (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('agemoea2 (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('agemoea2 (Colt): ' + str(i_Colt) + '\n')
    f.write('agemoea2 (Funet): ' + str(i_Funet) + '\n')
    f.write('agemoea2 (Abvt): ' + str(i_Abvt) + '\n')
    f.write('agemoea2 (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('agemoea2 (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('agemoea2 (Internode): ' + str(i_Internode) + '\n')
    f.write('agemoea2 (Missouri): ' + str(i_Missouri) + '\n')
    f.write('agemoea2 (Ion): ' + str(i_Ion) + '\n')
    f.write('agemoea2 (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('agemoea2 (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('agemoea2 (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('agemoea2 (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('agemoea2 (Colt): ' + str(n_Colt) + '\n')
    f1.write('agemoea2 (Funet): ' + str(n_Funet) + '\n')
    f1.write('agemoea2 (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('agemoea2 (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('agemoea2 (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('agemoea2 (Internode): ' + str(n_Internode) + '\n')
    f1.write('agemoea2 (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('agemoea2 (Ion): ' + str(n_Ion) + '\n')
    f1.write('agemoea2 (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For agemoea2 method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For agemoea2 method: 
Ideal point:  [2.00000000e+00 3.00000000e+00 5.23276816e+01 5.88159667e-03]
Nadir point:  [1.34000000e+02 1.20000000e+01 1.16714796e+03 5.62392153e-01]


## Load nsga2 results

In [14]:
with open('res_bc_Cogent_nsga2.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_nsga2.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_nsga2.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_nsga2.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_nsga2.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_nsga2.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_nsga2.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_nsga2.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_nsga2.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_nsga2.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_nsga2.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_nsga2.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [15]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('nsga2 (Cogent): ' + str(i_Cogent) + '\n')
    f.write('nsga2 (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('nsga2 (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('nsga2 (Colt): ' + str(i_Colt) + '\n')
    f.write('nsga2 (Funet): ' + str(i_Funet) + '\n')
    f.write('nsga2 (Abvt): ' + str(i_Abvt) + '\n')
    f.write('nsga2 (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('nsga2 (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('nsga2 (Internode): ' + str(i_Internode) + '\n')
    f.write('nsga2 (Missouri): ' + str(i_Missouri) + '\n')
    f.write('nsga2 (Ion): ' + str(i_Ion) + '\n')
    f.write('nsga2 (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('nsga2 (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('nsga2 (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('nsga2 (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('nsga2 (Colt): ' + str(n_Colt) + '\n')
    f1.write('nsga2 (Funet): ' + str(n_Funet) + '\n')
    f1.write('nsga2 (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('nsga2 (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('nsga2 (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('nsga2 (Internode): ' + str(n_Internode) + '\n')
    f1.write('nsga2 (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('nsga2 (Ion): ' + str(n_Ion) + '\n')
    f1.write('nsga2 (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For nsga2 method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For nsga2 method: 
Ideal point:  [2.00000000e+00 3.00000000e+00 5.22948653e+01 3.98424016e-03]
Nadir point:  [1.79000000e+02 7.70000000e+01 2.77917731e+03 8.63002054e-01]


## Load nsga3 results

In [16]:
with open('res_bc_Cogent_nsga3.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_nsga3.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_nsga3.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_nsga3.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_nsga3.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_nsga3.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_nsga3.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_nsga3.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_nsga3.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_nsga3.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_nsga3.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_nsga3.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [17]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('nsga3 (Cogent): ' + str(i_Cogent) + '\n')
    f.write('nsga3 (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('nsga3 (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('nsga3 (Colt): ' + str(i_Colt) + '\n')
    f.write('nsga3 (Funet): ' + str(i_Funet) + '\n')
    f.write('nsga3 (Abvt): ' + str(i_Abvt) + '\n')
    f.write('nsga3 (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('nsga3 (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('nsga3 (Internode): ' + str(i_Internode) + '\n')
    f.write('nsga3 (Missouri): ' + str(i_Missouri) + '\n')
    f.write('nsga3 (Ion): ' + str(i_Ion) + '\n')
    f.write('nsga3 (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('nsga3 (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('nsga3 (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('nsga3 (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('nsga3 (Colt): ' + str(n_Colt) + '\n')
    f1.write('nsga3 (Funet): ' + str(n_Funet) + '\n')
    f1.write('nsga3 (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('nsga3 (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('nsga3 (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('nsga3 (Internode): ' + str(n_Internode) + '\n')
    f1.write('nsga3 (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('nsga3 (Ion): ' + str(n_Ion) + '\n')
    f1.write('nsga3 (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For nsga3 method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For nsga3 method: 
Ideal point:  [2.00000000e+00 3.00000000e+00 5.45634444e+01 9.52115249e-03]
Nadir point:  [1.02000000e+02 1.80000000e+01 1.03037170e+03 5.62364256e-01]


## Load rnsga2 results

In [18]:
with open('res_bc_Cogent_rnsga2.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_rnsga2.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_rnsga2.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_rnsga2.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_rnsga2.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_rnsga2.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_rnsga2.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_rnsga2.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_rnsga2.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_rnsga2.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_rnsga2.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_rnsga2.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [19]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('rnsga2 (Cogent): ' + str(i_Cogent) + '\n')
    f.write('rnsga2 (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('rnsga2 (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('rnsga2 (Colt): ' + str(i_Colt) + '\n')
    f.write('rnsga2 (Funet): ' + str(i_Funet) + '\n')
    f.write('rnsga2 (Abvt): ' + str(i_Abvt) + '\n')
    f.write('rnsga2 (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('rnsga2 (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('rnsga2 (Internode): ' + str(i_Internode) + '\n')
    f.write('rnsga2 (Missouri): ' + str(i_Missouri) + '\n')
    f.write('rnsga2 (Ion): ' + str(i_Ion) + '\n')
    f.write('rnsga2 (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('rnsga2 (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('rnsga2 (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('rnsga2 (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('rnsga2 (Colt): ' + str(n_Colt) + '\n')
    f1.write('rnsga2 (Funet): ' + str(n_Funet) + '\n')
    f1.write('rnsga2 (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('rnsga2 (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('rnsga2 (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('rnsga2 (Internode): ' + str(n_Internode) + '\n')
    f1.write('rnsga2 (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('rnsga2 (Ion): ' + str(n_Ion) + '\n')
    f1.write('rnsga2 (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For rnsga2 method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For rnsga2 method: 
Ideal point:  [2.00000000e+00 3.00000000e+00 1.39471808e+02 1.27267197e-02]
Nadir point:  [6.80000000e+01 6.30000000e+01 2.33340309e+03 8.65002149e-01]


## Load rnsga3 results

In [20]:
with open('res_bc_Cogent_rnsga3.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_rnsga3.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_rnsga3.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_rnsga3.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_rnsga3.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_rnsga3.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_rnsga3.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_rnsga3.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_rnsga3.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_rnsga3.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_rnsga3.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_rnsga3.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [21]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('rnsga3 (Cogent): ' + str(i_Cogent) + '\n')
    f.write('rnsga3 (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('rnsga3 (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('rnsga3 (Colt): ' + str(i_Colt) + '\n')
    f.write('rnsga3 (Funet): ' + str(i_Funet) + '\n')
    f.write('rnsga3 (Abvt): ' + str(i_Abvt) + '\n')
    f.write('rnsga3 (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('rnsga3 (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('rnsga3 (Internode): ' + str(i_Internode) + '\n')
    f.write('rnsga3 (Missouri): ' + str(i_Missouri) + '\n')
    f.write('rnsga3 (Ion): ' + str(i_Ion) + '\n')
    f.write('rnsga3 (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('rnsga3 (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('rnsga3 (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('rnsga3 (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('rnsga3 (Colt): ' + str(n_Colt) + '\n')
    f1.write('rnsga3 (Funet): ' + str(n_Funet) + '\n')
    f1.write('rnsga3 (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('rnsga3 (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('rnsga3 (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('rnsga3 (Internode): ' + str(n_Internode) + '\n')
    f1.write('rnsga3 (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('rnsga3 (Ion): ' + str(n_Ion) + '\n')
    f1.write('rnsga3 (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For rnsga3 method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For rnsga3 method: 
Ideal point:  [4.00000000e+00 3.00000000e+00 5.60947590e+01 1.10292687e-02]
Nadir point:  [9.20000000e+01 1.10000000e+01 5.97970124e+02 1.97240135e-01]


## Load rvea results

In [22]:
with open('res_bc_Cogent_rvea.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_rvea.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_rvea.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_rvea.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_rvea.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_rvea.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_rvea.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_rvea.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_rvea.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_rvea.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_rvea.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_rvea.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [23]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('rvea (Cogent): ' + str(i_Cogent) + '\n')
    f.write('rvea (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('rvea (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('rvea (Colt): ' + str(i_Colt) + '\n')
    f.write('rvea (Funet): ' + str(i_Funet) + '\n')
    f.write('rvea (Abvt): ' + str(i_Abvt) + '\n')
    f.write('rvea (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('rvea (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('rvea (Internode): ' + str(i_Internode) + '\n')
    f.write('rvea (Missouri): ' + str(i_Missouri) + '\n')
    f.write('rvea (Ion): ' + str(i_Ion) + '\n')
    f.write('rvea (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('rvea (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('rvea (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('rvea (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('rvea (Colt): ' + str(n_Colt) + '\n')
    f1.write('rvea (Funet): ' + str(n_Funet) + '\n')
    f1.write('rvea (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('rvea (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('rvea (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('rvea (Internode): ' + str(n_Internode) + '\n')
    f1.write('rvea (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('rvea (Ion): ' + str(n_Ion) + '\n')
    f1.write('rvea (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For rvea method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For rvea method: 
Ideal point:  [1.00000000e+00 2.00000000e+00 5.41110348e+01 2.06189566e-02]
Nadir point:  [5.60000000e+01 4.60000000e+01 2.55626052e+03 1.70537209e+00]


## Load smsemoa results

In [24]:
with open('res_bc_Cogent_smsemoa.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_smsemoa.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_smsemoa.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_smsemoa.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_smsemoa.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_smsemoa.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_smsemoa.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_smsemoa.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_smsemoa.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_smsemoa.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_smsemoa.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_smsemoa.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [25]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('smsemoa (Cogent): ' + str(i_Cogent) + '\n')
    f.write('smsemoa (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('smsemoa (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('smsemoa (Colt): ' + str(i_Colt) + '\n')
    f.write('smsemoa (Funet): ' + str(i_Funet) + '\n')
    f.write('smsemoa (Abvt): ' + str(i_Abvt) + '\n')
    f.write('smsemoa (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('smsemoa (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('smsemoa (Internode): ' + str(i_Internode) + '\n')
    f.write('smsemoa (Missouri): ' + str(i_Missouri) + '\n')
    f.write('smsemoa (Ion): ' + str(i_Ion) + '\n')
    f.write('smsemoa (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('smsemoa (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('smsemoa (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('smsemoa (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('smsemoa (Colt): ' + str(n_Colt) + '\n')
    f1.write('smsemoa (Funet): ' + str(n_Funet) + '\n')
    f1.write('smsemoa (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('smsemoa (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('smsemoa (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('smsemoa (Internode): ' + str(n_Internode) + '\n')
    f1.write('smsemoa (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('smsemoa (Ion): ' + str(n_Ion) + '\n')
    f1.write('smsemoa (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For smsemoa method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For smsemoa method: 
Ideal point:  [2.00000000e+00 3.00000000e+00 5.40879082e+01 9.46486027e-03]
Nadir point:  [1.00000000e+02 1.30000000e+01 1.23716807e+03 5.62392153e-01]


## Load unsga3 results

In [26]:
with open('res_bc_Cogent_unsga3.pkl','rb') as f_Cogent:
    res_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_unsga3.pkl','rb') as f_UsCarrier:
    res_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_unsga3.pkl','rb') as f_HiberniaGlobal:
    res_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_unsga3.pkl','rb') as f_Colt:
    res_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_unsga3.pkl','rb') as f_Funet:
    res_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_unsga3.pkl','rb') as f_Abvt:
    res_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_unsga3.pkl','rb') as f_Intellifiber:
    res_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_unsga3.pkl','rb') as f_TataNld:
    res_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_unsga3.pkl','rb') as f_Internode:
    res_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_unsga3.pkl','rb') as f_Missouri:
    res_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_unsga3.pkl','rb') as f_Ion:
    res_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_unsga3.pkl','rb') as f_Palmetto:
    res_Palmetto = pickle.load(f_Palmetto)

In [27]:
F_Cogent = res_Cogent.F
F_UsCarrier = res_UsCarrier.F
F_HiberniaGlobal = res_HiberniaGlobal.F
F_Colt = res_Colt.F
F_Funet = res_Funet.F
F_Abvt = res_Abvt.F
F_Intellifiber = res_Intellifiber.F
F_TataNld = res_TataNld.F
F_Internode = res_Internode.F
F_Missouri = res_Missouri.F
F_Ion = res_Ion.F
F_Palmetto = res_Palmetto.F

i_Cogent = F_Cogent.min(axis=0)
n_Cogent = F_Cogent.max(axis=0)
i_UsCarrier = F_UsCarrier.min(axis=0)
n_UsCarrier = F_UsCarrier.max(axis=0)
i_HiberniaGlobal = F_HiberniaGlobal.min(axis=0)
n_HiberniaGlobal = F_HiberniaGlobal.max(axis=0)
i_Colt = F_Colt.min(axis=0)
n_Colt = F_Colt.max(axis=0)
i_Funet = F_Funet.min(axis=0)
n_Funet = F_Funet.max(axis=0)
i_Abvt = F_Abvt.min(axis=0)
n_Abvt = F_Abvt.max(axis=0)
i_Intellifiber = F_Intellifiber.min(axis=0)
n_Intellifiber = F_Intellifiber.max(axis=0)
i_TataNld = F_TataNld.min(axis=0)
n_TataNld = F_TataNld.max(axis=0)
i_Internode = F_Internode.min(axis=0)
n_Internode = F_Internode.max(axis=0)
i_Missouri = F_Missouri.min(axis=0)
n_Missouri = F_Missouri.max(axis=0)
i_Ion = F_Ion.min(axis=0)
n_Ion = F_Ion.max(axis=0)
i_Palmetto = F_Palmetto.min(axis=0)
n_Palmetto = F_Palmetto.max(axis=0)

with open('ips_of_all.txt', 'a', encoding='utf-8') as f:
    f.write('unsga3 (Cogent): ' + str(i_Cogent) + '\n')
    f.write('unsga3 (UsCarrier): ' + str(i_UsCarrier) + '\n')
    f.write('unsga3 (HiberniaGlobal): ' + str(i_HiberniaGlobal) + '\n')
    f.write('unsga3 (Colt): ' + str(i_Colt) + '\n')
    f.write('unsga3 (Funet): ' + str(i_Funet) + '\n')
    f.write('unsga3 (Abvt): ' + str(i_Abvt) + '\n')
    f.write('unsga3 (Intellifiber): ' + str(i_Intellifiber) + '\n')
    f.write('unsga3 (TataNld)): ' + str(i_TataNld) + '\n')
    f.write('unsga3 (Internode): ' + str(i_Internode) + '\n')
    f.write('unsga3 (Missouri): ' + str(i_Missouri) + '\n')
    f.write('unsga3 (Ion): ' + str(i_Ion) + '\n')
    f.write('unsga3 (Palmetto): ' + str(i_Palmetto) + '\n')    

with open('nps_of_all.txt', 'a', encoding='utf-8') as f1:
    f1.write('unsga3 (Cogent): ' + str(n_Cogent) + '\n')
    f1.write('unsga3 (UsCarrier): ' + str(n_UsCarrier) + '\n')
    f1.write('unsga3 (HiberniaGlobal): ' + str(n_HiberniaGlobal) + '\n')
    f1.write('unsga3 (Colt): ' + str(n_Colt) + '\n')
    f1.write('unsga3 (Funet): ' + str(n_Funet) + '\n')
    f1.write('unsga3 (Abvt): ' + str(n_Abvt) + '\n')
    f1.write('unsga3 (Intellifiber): ' + str(n_Intellifiber) + '\n')
    f1.write('unsga3 (TataNld)): ' + str(n_TataNld) + '\n')
    f1.write('unsga3 (Internode): ' + str(n_Internode) + '\n')
    f1.write('unsga3 (Missouri): ' + str(n_Missouri) + '\n')
    f1.write('unsga3 (Ion): ' + str(n_Ion) + '\n')
    f1.write('unsga3 (Palmetto): ' + str(n_Palmetto) + '\n')    

ideal_list = []
nadir_list = []
ideal_list.append(i_Cogent)
ideal_list.append(i_UsCarrier)
ideal_list.append(i_HiberniaGlobal)
ideal_list.append(i_Colt)
ideal_list.append(i_Funet)
ideal_list.append(i_Abvt)
ideal_list.append(i_Intellifiber)
ideal_list.append(i_TataNld)
ideal_list.append(i_Internode)
ideal_list.append(i_Missouri)
ideal_list.append(i_Ion)
ideal_list.append(i_Palmetto)

nadir_list.append(n_Cogent)
nadir_list.append(n_UsCarrier)
nadir_list.append(n_HiberniaGlobal)
nadir_list.append(n_Colt)
nadir_list.append(n_Funet)
nadir_list.append(n_Abvt)
nadir_list.append(n_Intellifiber)
nadir_list.append(n_TataNld)
nadir_list.append(n_Internode)
nadir_list.append(n_Missouri)
nadir_list.append(n_Ion)
nadir_list.append(n_Palmetto)

ideal_point = np.min(ideal_list, axis=0)
nadir_point = np.max(nadir_list, axis=0)

print("For unsga3 method: ")
print("Ideal point: ", ideal_point)
print("Nadir point: ", nadir_point)

For unsga3 method: 
Ideal point:  [2.00000000e+00 3.00000000e+00 5.39326977e+01 9.22699472e-03]
Nadir point:  [9.90000000e+01 1.90000000e+01 9.24468178e+02 5.50957737e-01]


## Final result

In [1]:
import numpy as np
final_nadir_point_list = []

# For ctaea method: 
final_nadir_point_list.append([9.90000000e+01, 5.00000000e+01, 9.56751065e+02, 5.62449763e-01])
# For dnsga2 method: 
# final_nadir_point_list.append([177.,  67.,  np.inf,  np.inf])
# # For kgbdmoea method: 
# final_nadir_point_list.append([163.,  73.,  np.inf,  np.inf])
# For moead method: 
final_nadir_point_list.append([4.50000000e+01, 6.00000000e+00, 4.79792713e+02, 1.33567032e-01])
# For agemoea method: 
final_nadir_point_list.append([1.46000000e+02, 3.30000000e+01, 1.59670616e+03, 8.55748626e-01])
# For agemoea2 method: 
final_nadir_point_list.append([1.34000000e+02, 1.20000000e+01, 1.16714796e+03, 5.62392153e-01])
# For nsga2 method: 
final_nadir_point_list.append([1.79000000e+02, 7.70000000e+01, 2.77917731e+03, 8.63002054e-01])
# For nsga3 method: 
final_nadir_point_list.append([1.02000000e+02, 1.80000000e+01, 1.03037170e+03, 5.62364256e-01])
# For rnsga2 method: 
final_nadir_point_list.append([6.80000000e+01, 6.30000000e+01, 2.33340309e+03, 8.65002149e-01])
# For rnsga3 method: 
final_nadir_point_list.append([9.20000000e+01, 1.10000000e+01, 5.97970124e+02, 1.97240135e-01])
# For smsemoa method: 
final_nadir_point_list.append([1.00000000e+02, 1.30000000e+01, 1.23716807e+03, 5.62392153e-01])
# For unsga3 method: 
final_nadir_point_list.append([9.90000000e+01, 1.90000000e+01, 9.24468178e+02, 5.50957737e-01])

final_nadir_point = np.max(final_nadir_point_list, axis=0)
print(final_nadir_point)

[1.79000000e+02 7.70000000e+01 2.77917731e+03 8.65002149e-01]
